## DeepDream Image Modification

In [ ]:
# load pretrained inception v3 model
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K
import tensorflow as tf
    
tf.compat.v1.disable_eager_execution()

K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

In [47]:
# create loss from summing the weighted l2 norm of the activations of each layer
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.0,
    'mixed4': 2.0,
    'mixed5': 1.5
}

loss = K.variable(0.0)

layer_dict = dict([(layer.name, layer) for layer in model.layers])
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output[:, 2:-2, 2:-2, :] # layer output of non-border pixels
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + (coeff * K.sum(K.square(activation)) / scaling)

In [48]:
# implement gradient ascent
dream_image = model.input

grads = K.gradients(loss, dream_image)[0] # gradient of dream_image wrt loss
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # normalize grads

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream_image], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ': ', loss_value)
        x += step * grad_values
    return x

In [49]:
# auxillary functions
import scipy
import numpy as np
from tensorflow.keras.preprocessing import image
import imageio

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.0
    x += 0.5
    x *= 225.0
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [50]:
# run gradient ascent at different octaves (successively larger scales)
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.0

base_image_path = 'creative_commons_elephant.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations, step, max_loss)
    
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

Processing image shape (489, 734)


2024-01-08 00:02:52.689192: W tensorflow/c/c_api.cc:305] Operation '{name:'Variable_6/Assign' id:12476 op device:{requested: '', assigned: ''} def:{{{node Variable_6/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](Variable_6, Variable_6/Initializer/initial_value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


...Loss value at 0 :  1.7700328
...Loss value at 1 :  2.3979902
...Loss value at 2 :  3.1967382
...Loss value at 3 :  4.164144
...Loss value at 4 :  5.190815
...Loss value at 5 :  6.2296495
...Loss value at 6 :  7.24692
...Loss value at 7 :  8.301004
...Loss value at 8 :  9.31803
Processing image shape (685, 1028)
...Loss value at 0 :  3.6032002
...Loss value at 1 :  4.9887047
...Loss value at 2 :  6.1564927
...Loss value at 3 :  7.274338
...Loss value at 4 :  8.358616
...Loss value at 5 :  9.393939
Processing image shape (960, 1440)
...Loss value at 0 :  3.6558778
...Loss value at 1 :  4.9381976
...Loss value at 2 :  6.050712
...Loss value at 3 :  7.135713
...Loss value at 4 :  8.22107
...Loss value at 5 :  9.26264
